In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Url'].tolist()

总数量：1266


['https://portal.thepartshq.com/marmon-ride-control-a2039-manual-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2073-manual-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2107-automatic-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2143-automatic-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2162-manual-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2163-manual-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2172-automatic-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2201-manual-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2217-manual-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2236-automatic-transmission-mount.html',
 'https://portal.thepartshq.com/marmon-ride-control-a2253-automatic-transmissi

In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get(input_.loc[a, 'Url'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

SSLError: HTTPSConnectionPool(host='portal.thepartshq.com', port=443): Max retries exceeded with url: /marmon-ride-control-a2039-manual-transmission-mount.html (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1002)')))

In [ ]:
from bs4 import BeautifulSoup
from lxml import etree

In [ ]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./oe.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

In [ ]:
list_tr = []
list_div = html.xpath('//div[@class="product info detailed"]/div/div')
for i in range(len(list_div)):
    if len(list_div[i].xpath('./a/text()')) != 0 and list_div[i].xpath('./a/text()')[0].strip() == 'Cross Reference':
        list_tr = list_div[i+1].xpath('./table/tbody/tr')
        
list_tr

In [ ]:
df_temp = pd.DataFrame([{'Url': input_.loc[a, 'Url']}])

for tr in list_tr:
    df_temp.loc[0, 'OE_' + tr.xpath('./td[1]/text()')[0].strip()] = tr.xpath('./td[2]/text()')[0].strip()
    
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

In [ ]:
crawler_status = 'ok'

print(crawler_status)

In [ ]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Url': input_.loc[a, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

In [ ]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_oe.xlsx', index=False)

output_correct

In [ ]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_oe_error.xlsx', index=False)

output_error